# interfaces

> This module defines the API and the GUI of *findmycells*

In [ ]:
#| default_exp interfaces

In [ ]:
#| export

from abc import ABC, abstractmethod
from pathlib import Path, PosixPath
from typing import List, Dict, Tuple, Optional, Union
from traitlets.traitlets import MetaHasTraits as WidgetType

import os
import pickle
import random
import pandas as pd
from datetime import datetime
import ipywidgets as w
from IPython.display import display
from ipyfilechooser import FileChooser
from tqdm.notebook import tqdm

from findmycells.configs import ProjectConfigs
from findmycells.database import Database
from findmycells.core import ProcessingStrategy, ProcessingObject
from findmycells.preprocessing.specs import PreprocessingStrategy, PreprocessingObject
from findmycells.segmentation.specs import SegmentationStrategy, SegmentationObject
from findmycells.postprocessing.specs import PostprocessingStrategy, PostprocessingObject
from findmycells.quantification.specs import QuantificationStrategy, QuantificationObject

In [ ]:
#| hide
from nbdev.showdoc import *

# Application programming interface (API)

The following class defines the API of findmycells, which represents the intended way of how users interact with and use findmycells, if they are not using the graphical user interface (GUI; defined below).

In [ ]:
#| export

class API:
    
    
    def __init__(self, project_root_dir: PosixPath) -> None:
        assert type(project_root_dir) == PosixPath, '"project_root_dir" must be pathlib.Path object referring to an existing directory.'
        assert project_root_dir.is_dir(), '"project_root_dir" must be pathlib.Path object referring to an existing directory.'
        self.project_configs = ProjectConfigs(root_dir = project_root_dir)
        self.database = Database(project_configs = self.project_configs)
        
    
    def update_database_with_current_source_files(self) -> None:
        self.database.compute_file_infos()
        
        
    def set_microscopy_reader_configs(self,
                                      microscopy_reader_configs: Optional[Dict]=None
                                     ) -> None:
        microscopy_reader_configs = self._assert_and_update_reader_configs_input(reader_type = 'microscopy_images',
                                                                                 reader_configs = microscopy_reader_configs)
        self.project_configs.add_reader_configs(reader_type = 'microscopy_images', reader_configs = microscopy_reader_configs)
        
        
    def set_roi_reader_configs(self,
                               roi_reader_configs: Optional[Dict]=None
                              ) -> None:
        roi_reader_configs = self._assert_and_update_reader_configs_input(reader_type = 'rois', reader_configs = roi_reader_configs)
        self.project_configs.add_reader_configs(reader_type = 'rois', reader_configs = roi_reader_configs)
    
    
    
    def save_status(self) -> None:
        date = f'{datetime.now():%Y_%m_%d}'
        dbase_filename = f'{date}_findmycells_database.dbase'
        self._save_attr_to_disk(attr_id = 'database', filename = dbase_filename, child_attr_ids_to_del = ['project_configs'])
        configs_filename = f'{date}_findmycells_project.configs'
        self._save_attr_to_disk(attr_id = 'project_configs', filename = configs_filename, child_attr_ids_to_del = ['available_processing_modules'])
        
        
    def load_status(self,
                    project_configs_filepath: Optional[PosixPath]=None,
                    database_filepath: Optional[PosixPath]=None
                   ) -> None:
        if project_configs_filepath != None:
            assert type(project_configs_filepath) == PosixPath, '"project_configs_filepath" must be pathlib.Path object referring to a .configs file.'
            assert project_configs_filepath.suffix == '.configs', '"project_configs_filepath" must be pathlib.Path object referring to a .configs file.'
        else:
            project_configs_filepath = self._look_for_latest_status_file_in_dir(suffix = '.configs', dir_path = self.project_configs.root_dir)
        if database_filepath != None:
            assert type(database_filepath) == PosixPath, '"database_filepath" must be pathlib.Path object referring to a .dbase file'
            assert database_filepath.suffix == '.dbase', '"database_filepath" must be pathlib.Path object referring to a .dbase file'
        else:
            database_filepath = self._look_for_latest_status_file_in_dir(suffix = '.dbase', dir_path = self.project_configs.root_dir)
        if hasattr(self, 'project_configs'):
            delattr(self, 'project_configs')
        if hasattr(self, 'database'):
            delattr(self, 'database')
        self.project_configs = self._load_object_from_filepath(filepath = project_configs_filepath)
        self.project_configs.load_available_processing_modules()
        self.database = self._load_object_from_filepath(filepath = database_filepath)
        setattr(self.database, 'project_configs', self.project_configs)
        
        
    def preprocess(self,
                   strategies: List[PreprocessingStrategy],
                   strategy_configs: Optional[List[Dict]]=None,
                   processing_configs: Optional[Dict]=None,
                   file_ids: Optional[List[str]]=None
                  ) -> None:
        processing_step_id = 'preprocessing'
        strategy_configs, processing_configs, file_ids = self._assert_and_update_input(processing_step_id = processing_step_id,
                                                                                       strategies = strategies,
                                                                                       strategy_configs = strategy_configs,
                                                                                       processing_configs = processing_configs,
                                                                                       file_ids = file_ids)
        self._assert_reader_configs_are_present()
        microscopy_reader_configs = getattr(self.project_configs, 'microscopy_images')
        roi_reader_configs = getattr(self.project_configs, 'rois')
        for file_id in tqdm(file_ids, display = processing_configs['show_progress']):
            preprocessing_object = PreprocessingObject()
            preprocessing_object.prepare_for_processing(file_ids = [file_id], database = self.database)
            preprocessing_object.load_image_and_rois(microscopy_reader_configs = microscopy_reader_configs, roi_reader_configs = roi_reader_configs)
            preprocessing_object.run_all_strategies(strategies = strategies, strategy_configs = strategy_configs)
            preprocessing_object.save_preprocessed_images_on_disk()
            preprocessing_object.save_preprocessed_rois_in_database()
            preprocessing_object.update_database(mark_as_completed = True)
            del preprocessing_object
            if processing_configs['autosave'] == True:
                self.save_status()
                self.load_status()    
    
    
    def segment(self,
                   strategies: List[SegmentationStrategy],
                   strategy_configs: Optional[List[Dict]]=None,
                   processing_configs: Optional[Dict]=None,
                   file_ids: Optional[List[str]]=None
                  ) -> None:
        processing_step_id = 'segmentation'
        strategy_configs, processing_configs, file_ids = self._assert_and_update_input(processing_step_id = processing_step_id,
                                                                                       strategies = strategies,
                                                                                       strategy_configs = strategy_configs,
                                                                                       processing_configs = processing_configs,
                                                                                       file_ids = file_ids)
        file_ids_per_batch = self._split_file_ids_into_batches(file_ids = file_ids, batch_size = processing_configs['batch_size'])
        if processing_configs['run_strategies_individually'] == True:
            self._segment_running_strategies_individually(strategies = strategies,
                                                          strategy_configs = strategy_configs,
                                                          processing_configs = processing_configs,
                                                          file_ids_per_batch = file_ids_per_batch)
        else:
            self._segment_running_strategies_consecutively(strategies = strategies,
                                                           strategy_configs = strategy_configs,
                                                           processing_configs = processing_configs,
                                                           file_ids_per_batch = file_ids_per_batch)
        if processing_configs['clear_tmp_data'] == True:
            all_files_done = self._check_if_all_files_have_finished_current_processing_step(processing_step_id = processing_step_id)
            if all_files_done == True:
                segmentation_object = SegmentationObject()
                dummy_file_id = file_ids_per_batch[0][0]
                segmentation_object.prepare_for_processing(file_ids = [dummy_file_id], database = self.database)
                segmentation_object.clear_all_tmp_data_in_seg_tool_dir()
                del segmentation_object    
    
    
    def postprocess(self,
                    strategies: List[PostprocessingStrategy],
                    strategy_configs: Optional[List[Dict]]=None,
                    processing_configs: Optional[Dict]=None,
                    file_ids: Optional[List[str]]=None
                   ) -> None:
        processing_step_id = 'postprocessing'
        strategy_configs, processing_configs, file_ids = self._assert_and_update_input(processing_step_id = processing_step_id,
                                                                                       strategies = strategies,
                                                                                       strategy_configs = strategy_configs,
                                                                                       processing_configs = processing_configs,
                                                                                       file_ids = file_ids)
        for file_id in tqdm(file_ids, display = processing_configs['show_progress']):
            postprocessing_object = PostprocessingObject()
            postprocessing_object.prepare_for_processing(file_ids = [file_id], database = self.database)
            postprocessing_object.load_segmentations_masks_for_postprocessing(segmentations_to_use = processing_configs['segmentations_to_use'])
            postprocessing_object.run_all_strategies(strategies = strategies, strategy_configs = strategy_configs)
            postprocessing_object.save_postprocessed_segmentations()
            postprocessing_object.update_database(mark_as_completed = True)
            del postprocessing_object
            if processing_configs['autosave'] == True:
                self.save_status()
                self.load_status()    
    
    
    def quantify(self,
                 strategies: List[QuantificationStrategy],
                 strategy_configs: Optional[List[Dict]]=None,
                 processing_configs: Optional[Dict]=None,#
                 file_ids: Optional[List[str]]=None
                ) -> None:
        processing_step_id = 'quantification'
        strategy_configs, processing_configs, file_ids = self._assert_and_update_input(processing_step_id = processing_step_id,
                                                                                       strategies = strategies,
                                                                                       strategy_configs = strategy_configs,
                                                                                       processing_configs = processing_configs,
                                                                                       file_ids = file_ids)
        for file_id in tqdm(file_ids, display = processing_configs['show_progress']):
            quantification_object = QuantificationObject()
            quantification_object.prepare_for_processing(file_ids = [file_id], database = self.database)
            quantification_object.run_all_strategies(strategies = strategies, strategy_configs = strategy_configs)
            quantification_object.update_database(mark_as_completed = True)
            del quantification_object
            if processing_configs['autosave'] == True:
                self.save_status()
                self.load_status()
                
                
    def initialize_inspection(self,
                              inspection_method_class: Inspection_Method,
                              file_id: str,
                              area_roi_id: str
                             ) -> InspectionMethod:
        inspection_method_obj = inspection_method_class()
        inspection_method_obj.load_data(file_id = file_id, area_roi_id = area_roi_id, database = self.database)
        return inspection_method_obj
    
    
    def inspect(self,
                inspection_method_obj: Inspection_Method,
                center_coords: Tuple[int, int],
                inspection_configs: Dict[str, Any]
               ) -> None:
        inspection_method_obj.run_inspection(center_coords = center_coords, inspection_configs = inspection_configs)
    

    def _assert_reader_configs_are_present(self) -> None:
        assert_message = ('You have to specify your {} reader configs first before running ".preprocess()".'
                          'You can do this by running the ".set_{}_reader_configs()" method first!')
        assert hasattr(self.project_configs, 'microscopy_images'), assert_message.format('microscopy image', 'microscopy')
        assert hasattr(self.project_configs, 'rois'), assert_message.format('ROI', 'roi')
                
                
    def _assert_and_update_reader_configs_input(self, reader_type: str, reader_configs: Optional[Dict]) -> Dict:            
        if reader_configs == None:
            if hasattr(self.project_configs, reader_type) == False:
                self.project_configs.add_reader_configs(reader_type = reader_type)
            reader_configs = getattr(self.project_configs, reader_type)
        else:
            assert type(reader_configs) == dict, f'"reader_configs" (data type: {reader_type}) has to be a dictionary!'
            default_configs = self.project_configs.data_reader_default_configs[reader_type]
            default_configs.assert_user_input(user_input = reader_configs)
            reader_configs = default_configs.fill_user_input_with_defaults_where_needed(user_input = reader_configs)
        return reader_configs
    
   
    def _segment_running_strategies_individually(self,
                                                 strategies: List[SegmentationStrategy],
                                                 strategy_configs: List[Dict],
                                                 processing_configs: Dict,
                                                 file_ids_per_batch: List[List[str]]
                                                ) -> None:
        total_strategy_count = len(strategies)
        for i in tqdm(range(total_strategy_count), display = processing_configs['show_progress']):
            if processing_configs['show_progress'] == True:
                print(f'Starting with segmentation strategy #{i+1}')
            strategy, config = strategies[i], strategy_configs[i]
            for batch_file_ids in tqdm(file_ids_per_batch, display = processing_configs['show_progress']):
                if processing_configs['show_progress'] == True:
                    print(f'Starting with batch #{file_ids_per_batch.index(batch_file_ids) + 1}')
                segmentation_object = SegmentationObject()
                segmentation_object.prepare_for_processing(file_ids = batch_file_ids, database = self.database)
                segmentation_object.run_all_strategies(strategies = [strategy], strategy_configs = [config])
                if i == total_strategy_count - 1: # if this is the last strategy that needs to be run
                    segmentation_object.update_database(mark_as_completed = True)
                else:
                    segmentation_object.update_database(mark_as_completed = False)
                del segmentation_object
                if processing_configs['autosave'] == True:
                    self.save_status()
                    self.load_status()


    def _segment_running_strategies_consecutively(self,
                                                  strategies: List[SegmentationStrategy],
                                                  strategy_configs: List[Dict],
                                                  processing_configs: Dict,
                                                  file_ids_per_batch: List[List[str]]
                                                 ) -> None:
        for batch_file_ids in tqdm(file_ids_per_batch, display = processing_configs['show_progress']):
            segmentation_object = SegmentationObject()
            segmentation_object.prepare_for_processing(file_ids = batch_file_ids, database = self.database)
            segmentation_object.run_all_strategies(strategies = strategies, strategy_configs = strategy_configs)
            segmentation_object.update_database(mark_as_completed = True)
            del segmentation_object
            if processing_configs['autosave'] == True:
                self.save_status()
                self.load_status()
                

    def _check_if_all_files_have_finished_current_processing_step(self, processing_step_id: str) -> bool:
        all_file_ids = self.database.file_infos['file_id']
        file_ids_not_processed_yet = []
        for file_id in all_file_ids:
            if processing_step_id not in self.database.file_histories[file_id].completed_processing_steps.keys():
                file_ids_not_processed_yet.append(file_id)
            else:
                if self.database.file_histories[file_id].completed_processing_steps[processing_step_id] == False:
                    file_ids_not_processed_yet.append(file_id)
        return len(file_ids_not_processed_yet) == 0

        
    
    def _save_attr_to_disk(self, attr_id: str, filename: str, child_attr_ids_to_del: List[str]) -> None:
        filepath = self.project_configs.root_dir.joinpath(filename)
        attribute_to_save = getattr(self, attr_id)
        for attr_id_to_del in child_attr_ids_to_del:
            delattr(attribute_to_save, attr_id_to_del)
        filehandler = open(filepath, 'wb')
        pickle.dump(attribute_to_save, filehandler)

        
    def _load_object_from_filepath(self, filepath: PosixPath) -> Union[Database, ProjectConfigs]:
        filehandler = open(filepath, 'rb')
        loaded_object = pickle.load(filehandler)
        return loaded_object
        

    def _split_file_ids_into_batches(self, file_ids: List[str], batch_size: int) -> List[List[str]]:
        """
        Splits a list ("file_ids") of file_id strings into nested lists of file_id strings,
        where the maximum length of each nested list equals the integer passed as "batch_size".
        If "batch_size" matches or exceeds the number of file_id strings passed in the original
        list (i.e. length of "file_ids"), or if "batch_size" is 0, it will return a list with 
        only a single nested list that again contains all file_id strings.
        """
        if batch_size == 0:
            file_ids_per_batch = [file_ids]
        else:
            if len(file_ids) % batch_size == 0:
                total_batches = int(len(file_ids) / batch_size)
            else:
                total_batches = int(len(file_ids) / batch_size) + 1
            file_ids_per_batch = []
            for batch in range(total_batches):
                if len(file_ids) >= batch_size:
                    sampled_file_ids = random.sample(file_ids, batch_size)
                else:
                    sampled_file_ids = file_ids.copy()
                file_ids_per_batch.append(sampled_file_ids)
                for elem in sampled_file_ids:
                    file_ids.remove(elem)    
        return file_ids_per_batch


    def _look_for_latest_status_file_in_dir(self, suffix: str, dir_path: PosixPath) -> PosixPath:
        matching_filepaths = [filepath for filepath in dir_path.iterdir() if filepath.suffix == suffix]
        if len(matching_filepaths) == 0:
            raise FileNotFoundError(f'Could not find a "{suffix}" file in {dir_path}. Consider specifying the exact filepath!')
        else:
            date_strings = [filepath.name[:10] for filepath in matching_filepaths]
            dates = [datetime.strptime(date_str, '%Y_%m_%d') for date_str in date_strings]
            latest_date = max(dates)
            filepath_idx = dates.index(latest_date)
            latest_status_filepath = matching_filepaths[filepath_idx]
        return latest_status_filepath        
        
        
    def _assert_and_update_input(self, 
                                 processing_step_id: str,
                                 strategies: List[PreprocessingStrategy],
                                 strategy_configs: Optional[List[Dict]],
                                 processing_configs: Optional[Dict],
                                 file_ids: Optional[List[str]]
                                ) -> Tuple[List[Dict], Dict, List[str]]:
        self._assert_processing_step_input(processing_step_id = processing_step_id,
                                           strategies = strategies,
                                           strategy_configs = strategy_configs,
                                           processing_configs = processing_configs,
                                           file_ids = file_ids)
        strategy_configs = self._fill_strategy_configs_with_defaults_where_needed(strategies, strategy_configs)
        if processing_configs == None:
            if hasattr(self.project_configs, processing_step_id) == False:
                self.project_configs.add_processing_step_configs(processing_step_id = processing_step_id)
            processing_configs = getattr(self.project_configs, processing_step_id)
        processing_configs = self._fill_processing_configs_with_defaults_where_needed(processing_step_id, processing_configs)
        self.project_configs.add_processing_step_configs(processing_step_id, configs = processing_configs)
        file_ids = self.database.get_file_ids_to_process(input_file_ids = file_ids,
                                                         processing_step_id = processing_step_id,
                                                         overwrite = processing_configs['overwrite'])
        return strategy_configs, processing_configs, file_ids
            
        
    def _assert_processing_step_input(self, 
                                      processing_step_id: str,
                                      strategies: List[PreprocessingStrategy],
                                      strategy_configs: Optional[List[Dict]],
                                      processing_configs: Optional[Dict],
                                      file_ids: Optional[List[str]]
                                     ) -> None:
        assert type(strategies) == list, '"strategies" has to be a list of ProcessingStrategy classes of the respective processing step!'
        if strategy_configs != None:
            assert type(strategy_configs) == list, '"strategy_configs" has to be None or a list of the same length as "strategies"!'
            assert len(strategy_configs) == len(strategies), '"strategy_configs" has to be None or a list of the same length as "strategies"!'
        else:
            strategy_configs = [None] * len(strategies)
        available_strategies = self.project_configs.available_processing_strategies[processing_step_id]
        for strat, config in zip(strategies, strategy_configs):
            assert strat in available_strategies, f'{strat} is not an available strategy for {processing_step_id}!'
            if config != None:
                strat().default_configs.assert_user_input(user_input = config)
        if processing_configs != None:
            processing_obj = self.project_configs.available_processing_objects[processing_step_id]()
            processing_obj.default_configs.assert_user_input(user_input = processing_configs)
        if file_ids != None:
            assert type(file_ids) == list, '"file_ids" has to be a list of strings referring to file_ids in the database!'
            for elem in file_ids:
                assert elem in self.database.file_infos['file_id'], f'{elem} is not a valid file_id!'
        
        
    def _fill_processing_configs_with_defaults_where_needed(self,
                                                            processing_step_id: str,
                                                            processing_configs: Dict
                                                           ) -> Dict:
        processing_obj = self.project_configs.available_processing_objects[processing_step_id]()
        return processing_obj.default_configs.fill_user_input_with_defaults_where_needed(user_input = processing_configs)                                              
             
        
    def _fill_strategy_configs_with_defaults_where_needed(self,
                                                          strategies: List[ProcessingStrategy],
                                                          strategy_configs: Optional[List[Dict]]
                                                         ) -> List[Dict]:
        all_final_configs = []
        if strategy_configs == None:
            for strat in strategies:
                default_configs = strat().default_configs.fill_user_input_with_defaults_where_needed(user_input = {})
                all_final_configs.append(default_configs)
        else:
            for strat, configs in zip(strategies, strategy_configs):
                full_configs = strat().default_configs.fill_user_input_with_defaults_where_needed(user_input = configs)
                all_final_configs.append(full_configs)
        return all_final_configs

# Graphical user interface

The following classes are used to create the graphical user interface of findmycells. Please note that classes will be listed here in an inverted hirarchy, such that you can find the main `GUI` class at the very end, and classes that handle much more specific details at the beginning.

In [ ]:
#| export

GUI_SPACER = w.Label(value = '', layout = {'height': '30px'})

In [ ]:
class StrategyConfigurator:
    
    """
    This class implements the interface that let´s the user choose and 
    configurate the processing strategies. It will be placed inside of
    an accordion that is implemented in the `ProcessingStepPage`.
    It gets a list of all available processing strategies from the parent
    `ProcessingStepPage` and, thus, eventually from the `API` that checks
    for all available processing strategies in the corresponding processing
    submodule (e.g. "findmycells.preprocessing.strategies"). Upon initializing
    it´s dropdown widget, which let´s the user browser through the different
    available strategies, it also initializes an object of each strategy.
    This object can then be used to run it´s associated method 
    ".initialize_gui_configs_and_widget()" to build the specified widget, 
    using its `GUIConfigs` instance. Essentially, this contains a
    description of what the processing strategy does and, if applicable,
    widgets to specify all parameters that can be configurated for this 
    strategy. Finally, a "confirm & export" and a "remove" button allow
    the user to load or delete the current to or from the findmycells
    project, respectively.
    """
    
    def __init__(self,
                 available_strategy_classes: List,
                 parent_accordion: w.Accordion,
                 target_for_configs_export: List) -> None:
        self.available_strategy_classes = available_strategy_classes
        self.parent_accordion = parent_accordion
        self.target_for_configs_export = target_for_configs_export
        self.widget = self._initialize_widget()
        self._link_widgets_with_eventhandlers()

        
    def _initialize_widget(self) -> WidgetType:
        info_text = w.HTML(value = ('Please select one of the available processing methods from the dropdown menu below. '
                                    'Feel free to click through all listed methods, as each of them will display a '
                                    'short description of what exactly to expect and may also prompt you with some '
                                    'customization options. To select a method with all selected customization options, '
                                    'click on the "confirm selection & export configurations". Using the "remove method" '
                                    'button allows you to remove previously loaded methods again.'))
        self.dropdown = self._initialize_dropdown()
        self.confirm_and_export_button = w.Button(description = 'confirm selection & export configurations', layout = {'width': '30%'})
        self.remove_button = w.Button(description = 'remove method', layout = {'width': '20%'}, disabled = True)
        self.displayed_strat_widget = w.VBox([self.dropdown.value.widget], layout = {'width': '95%'})
        widget = w.VBox([info_text,
                         GUI_SPACER,
                         w.HBox([self.dropdown, self.confirm_and_export_button, self.remove_button]),
                         GUI_SPACER,
                         self.displayed_strat_widget])
        return widget
        
        
    def _link_widgets_with_eventhandlers(self) -> None:
        self.confirm_and_export_button.on_click(self._confirm_and_export_button_clicked)
        self.remove_button.on_click(self._remove_button_clicked)
        self.dropdown.observe(self._dropdown_option_changed, names = 'value')
        
    
    def _initialize_dropdown(self) -> WidgetType:
        dropdown_option_tuples = []
        for strategy_class in self.available_strategy_classes:
            strategy_obj = strategy_class()
            strategy_obj.initialize_gui_configs_and_widget()
            dropdown_option_tuples.append((strategy_obj.dropdown_option_value_for_gui, strategy_obj))
        return w.Dropdown(options = dropdown_option_tuples, layout = {'width': '50%'}) 
        
        
    def _get_own_position_idx_in_parent_accordion(self) -> int:
        return self.parent_accordion.children.index(self.widget)
        
        
    def _confirm_and_export_button_clicked(self, b) -> None:
        self._export_configs()
        self._change_disable_settings_of_customizable_widgets(disable_customizable_widgets = True)
        self._add_new_strategy_configurator_to_parent_accordion()
        self._change_own_accordion_tab_title(title = self.dropdown.value.dropdown_option_value_for_gui)
        self.parent_accordion.selected_index = None
        
        
    def _add_new_strategy_configurator_to_parent_accordion(self) -> None:
        new_strategy_configurator = StrategyConfigurator(available_strategy_classes = self.available_strategy_classes,
                                                         parent_accordion = self.parent_accordion,
                                                         target_for_configs_export = self.target_for_configs_export)
        self.parent_accordion.children = self.parent_accordion.children + (new_strategy_configurator.widget, )
        position_idx = len(self.parent_accordion.children) - 1
        self.parent_accordion.set_title(position_idx, 'Expand me to add a processing method')
                                                         
                                                         
    def _change_own_accordion_tab_title(self, title: str) -> None:
        position_idx = self._get_own_position_idx_in_parent_accordion()
        self.parent_accordion.set_title(position_idx, title)

        
    def _export_configs(self) -> None:
        selected_strategy_obj = self.dropdown.value
        current_configs = selected_strategy_obj.gui_configs.export_current_config_values()
        position_idx = self._get_own_position_idx_in_parent_accordion()
        self.target_for_configs_export.insert(position_idx, (selected_strategy_obj.__class__, current_configs))      
        
        
    def _change_disable_settings_of_customizable_widgets(self, disable_customizable_widgets: bool) -> None:
        self.remove_button.disabled = not disable_customizable_widgets
        self.dropdown.disabled = disable_customizable_widgets
        self.confirm_and_export_button.disabled = disable_customizable_widgets
        for widget in self.displayed_strat_widget.children[0].children:
            if hasattr(widget, 'disabled'):
                widget.disabled = disable_customizable_widgets
            elif type(widget) == w.HBox:
                if hasattr(widget.children[0], 'disabled'):
                    widget.children[0].disabled = disable_customizable_widgets
        
        
    def _remove_button_clicked(self, b) -> None:
        self._remove_configs()
        if len(self.parent_accordion.children) == 1:
            self._change_disable_settings_of_customizable_widgets(disable_customizable_widgets = False)
            self._change_own_accordion_tab_title(title = 'Expand me to add a processing method')
        else:
            currently_present_accordion_tabs = len(self.parent_accordion.children)
            currently_confirmed_strategies = len(self.target_for_configs_export)
            tabs_available_for_selection = currently_present_accordion_tabs - currently_confirmed_strategies
            if tabs_available_for_selection > 1:
                self._remove_own_tab_from_parent_accordion()
            else:
                self._change_disable_settings_of_customizable_widgets(disable_customizable_widgets = False)
                self._change_own_accordion_tab_title(title = 'Expand me to add a processing method')
                
                
    def _remove_own_tab_from_parent_accordion(self) -> None:
        tmp_children = list(self.parent_accordion.children)
        tmp_titles = []
        for idx in range(len(tmp_children)):
            tmp_titles.append(self.parent_accordion.get_title(idx))
        position_idx = self._get_own_position_idx_in_parent_accordion()
        tmp_children.pop(position_idx)
        tmp_titles.pop(position_idx)
        self.parent_accordion.children = tuple(tmp_children)
        for idx, title in enumerate(tmp_titles):
            self.parent_accordion.set_title(idx, title)
            
            
    def _remove_configs(self) -> None:
        position_idx = self._get_own_position_idx_in_parent_accordion()
        self.target_for_configs_export.pop(position_idx)
                
                
    def _dropdown_option_changed(self, change) -> None:
        new_selection = change.new
        self.displayed_strat_widget.children = (new_selection.widget, )

In [ ]:
#| export

class PageButtonBundle(ABC):
    
    
    @abstractmethod
    def _initialize_page_content(self) -> WidgetType:
        pass
    
    
    def __init__(self, bundle_id: str, page_screen: WidgetType, all_navigator_buttons: List, api: API) -> None:
        self.bundle_id = bundle_id
        self.gui_page_screen = page_screen
        self.all_navigator_buttons = all_navigator_buttons
        self.api = api
        self.navigator_button = self._initialize_navigator_button()
        self.displayed_output = w.Output()
        self.page_content = self._initialize_page_content()
        
        
    def _initialize_navigator_button(self) -> WidgetType:
        navigator_button = w.Button(description = self.bundle_id, style = {'button_color': 'gray'})
        navigator_button.on_click(self._navigator_button_clicked)
        return navigator_button
    
    
    def _navigator_button_clicked(self, b) -> None:
        for button in self.all_navigator_buttons:
            button.style.button_color = 'gray'
        self.navigator_button.style.button_color = 'skyblue'
        self.gui_page_screen.children = (self.page_content, self.displayed_output)

In [ ]:
#| export

class SettingsPage(PageButtonBundle):
    
    """
    Subclass of `PageButtonBundle` that implements the GUI interface that allows the user to specify all 
    settings relevant to the findmycells project. It also enables saving & loading of the project status, 
    and to browse through the file history that is automatically created by findmycells.
    """
        
    def _initialize_page_content(self) -> WidgetType:
        settings_intro_text = w.HTML(value = ('This is the settings page for your findmycells '
                                              'project. You will find everything relevant regarding '
                                              'the organization of your project in the tabs below. '
                                              'If you have just started your findmycells project, '
                                              'please make sure to follow the instructions in the '
                                              '"project files" tab before you can get started.'))
        project_files_tab_widget = self._initialize_project_files_tab_widget()
        save_load_project_tab_widget = self._initialize_save_load_project_tab_widget()
        data_reader_tab_widget = self._initialize_data_reader_tab_widget()
        browse_file_histories_tab_widget = self._initialize_browse_file_histories_tab_widget()
        self._bind_buttons_to_functions()
        tabs = w.Tab([project_files_tab_widget, 
                      data_reader_tab_widget,
                      browse_file_histories_tab_widget,
                      save_load_project_tab_widget], selected_index = 0)
        tabs.set_title(0, 'project files')
        tabs.set_title(1, 'data import settings')
        tabs.set_title(2, 'browse file histories')
        tabs.set_title(3, 'save & load project')
        page_content_widget = w.VBox([settings_intro_text, tabs])
        return page_content_widget
        
        
    def _initialize_data_reader_tab_widget(self) -> WidgetType:
        intro_text = w.HTML(value = ('Please use the following widgets to specify the settigns of how '
                                     'data (i.e. microscopy images and, if available, ROI-files) shall '
                                     'be imported into your findmycells project:'))
        microscopy_images_reader_settings_widget = self._initialize_data_reader_settings_widget(reader_type = 'microscopy_images')
        rois_reader_settings_widget = self._initialize_data_reader_settings_widget(reader_type = 'rois')
        data_reader_accordion = w.Accordion([microscopy_images_reader_settings_widget, rois_reader_settings_widget])
        data_reader_accordion.set_title(0, 'Microscopy images import settings')
        data_reader_accordion.set_title(1, 'ROI-files import settings')
        return w.VBox([intro_text, data_reader_accordion])
        
        
    def _initialize_data_reader_settings_widget(self, reader_type: str) -> WidgetType:
        reader_specs = self.api.project_configs.available_data_readers[reader_type]()
        reader_specs.initialize_gui_configs_and_widget()
        confirm_reader_settings_button = w.Button(description = 'confirm settings')
        setattr(self, f'{reader_type}_reader_specs', reader_specs)
        setattr(self, f'confirm_{reader_type}_reader_settings_button', confirm_reader_settings_button)
        return w.VBox([reader_specs.widget, confirm_reader_settings_button])       
        
        
        
    def _initialize_project_files_tab_widget(self) -> WidgetType:
        intro_text = w.HTML(value = ('Just started a new project? Great! Before you can get started with '
                                     'the processing of your data, you need to associate the corresponding '
                                     'files with your findmycells project. Unfortunately, this requires you '
                                     'to arrange your files in a very rigid structure of directories (this '
                                     'will be fixed in a later version). But don´t worry, if you click on '
                                     'the "expand me if you need a detailed guide to prepare your data!" '
                                     'widget below you will find a comprehensive overview on how this tree '
                                     'of directories has to look like. Once you have your data arranged '
                                     'accordingly, just click the "update project files" button below. You '
                                     'can also always come back to this page & hit the button again in order '
                                     'to update the files associated with your project. Findmycells will then '
                                     'automatically identify files that have been deleted or added to the '
                                     'directories and remove them from or add them to your current project, '
                                     'respectively. To get an overview of which files are currently associ'
                                     'ated with your project, just click the "display current project files" '
                                     'button right next to the "update project files" button. If you are inter'
                                     'ested in more detailed information about each file, for instance its '
                                     'processing history, please head over to the "file histories" tab.'))
        guide_accordion = self._initialize_guide_accordion_widget()
        self.update_project_files_button = w.Button(description = 'update project files',
                                                    icon = 'refresh',
                                                    layout = {'width': '33%'})
        self.display_current_project_files_button = w.Button(description = 'display current project files',
                                                             layout = {'width': '33%'})
        buttons = w.HBox([self.update_project_files_button, self.display_current_project_files_button])
        self.current_project_files_output = w.Output()
        project_files_tab_widget = w.VBox([intro_text,
                                           guide_accordion,
                                           buttons,
                                           self.current_project_files_output])
        return project_files_tab_widget
    
    
    def _initialize_guide_accordion_widget(self) -> WidgetType:
        # ToDo: write detailed description of directory tree
        # ToDo: add sample image of how this could look like
        # ToDo: what to do for walkthrough through data
        # ToDo: hint that just clicking the button with an empty microscopy dir launches sample tree
        general_info = w.HTML(value = 'space for some general infos')
        detailed_infos = w.HTML(value = 'all details come here')
        sample_image = w.HTML(value = 'this shall be replaced by the sample image')
        whole_guide = w.VBox([general_info, 
                              detailed_infos,
                              sample_image])
        guide_accordion = w.Accordion([whole_guide], selected_index = None)
        guide_accordion.set_title(0, 'expand me if you need a detailed guide to prepare your data!')
        return guide_accordion
        
        
    def _initialize_save_load_project_tab_widget(self) -> WidgetType:
        intro_text = w.HTML(value = ('Here you can either save the progress of your currently '
                                     'running project or load a previously saved project.'))
        save_project_widget = self._initialize_save_project_widget()
        load_project_widget = self._initialize_load_project_widget()
        accordion = w.Accordion([save_project_widget, load_project_widget], selected_index = 0)
        accordion.set_title(0, 'save')
        accordion.set_title(1, 'load')
        save_load_project_tab_widget = w.VBox([intro_text, accordion])
        return save_load_project_tab_widget
    
    
    def _initialize_save_project_widget(self) -> WidgetType:
        save_description = w.HTML(value = ('Clicking the following "save" button will save '
                                           'your current project, including all configurations '
                                           'and processing progress. The file will automatically '
                                           'be written as a ".configs" file to the root directory '
                                           'you specified, with the current date as prefix.'))
        self.save_project_button = w.Button(description = 'save project', icon = 'save')
        save_project_widget = w.VBox([save_description, self.save_project_button])
        return save_project_widget
    
    
    def _initialize_load_project_widget(self) -> WidgetType:
        load_description = w.HTML(value = ('You already have a findmycells project to load? Great! '
                                           'Please make sure to choose the corresponding root dir'
                                           'rectory in which you previously created and run your '
                                           'project upon starting this GUI. If you have specified a '
                                           'different root directory, simply restart the GUI. When '
                                           'you are in the correct root directory, simply click the '
                                           '"load project" button to load your project and the last '
                                           'status you have saved.'))
        self.load_project_button = w.Button(description = 'load project', icon = 'upload')
        # ToDo: 
        #   For the moment you should only load a project from its own root dir.
        #   However, it should be possible in later versions to provide the filepath(s)
        #   to the file(s) that fmc created upon saving the project. This would, consequently,
        #   require the use of filechooser(s). I think it was possible to restrict the selection
        #   to only files with a specific extension (to make sure the user selects the correct
        #   file(s)). In addition, it might be possible to re-configure the default filepath
        #   of the current root dir. 
        load_project_widget = w.VBox([load_description,
                                      self.load_project_button])
        return load_project_widget
    
        
    def _initialize_browse_file_histories_tab_widget(self) -> WidgetType:
        """
        The options of the two dropdowns will be specified / updated when the following buttons are clicked: 
            - self.file_histories_id_dropdown: self.update_project_files_button
            - self.processing_step_id_dropdown: self.display_file_history_button
        The outputs created here will be used to display the following information:
            - self.file_infos_output: displays the file_infos (i.e. project level overview) of the selected file ID
            - self.file_history_output: displays the .tracked_history property of the corresponding FileHistory object of the selected file ID
            - self.processing_step_details_output: displays the detailed logs of the selected processing step
        """
        intro_text = w.HTML(value = ('Findmycells keeps a detailed track of how and when your '
                                     'files are processed. Using the widgets below, you are '
                                     'able to browser through this history for all files in your '
                                     'project.'))   
        self.file_histories_id_dropdown = w.Dropdown(description = 'History of file ID:', 
                                                     style = {'description_width': 'initial'},
                                                     layout = {'width': '66%'})
        self.display_file_history_button = w.Button(description = 'show history', layout = {'width': '33%'})
        dropdown_button_hbox = w.HBox([self.file_histories_id_dropdown, self.display_file_history_button])
        self.file_infos_output = w.Output()
        self.file_history_output = w.Output()
        genereal_file_history_widgets = w.VBox([w.HBox([self.file_histories_id_dropdown, self.display_file_history_button]),
                                                self.file_infos_output, 
                                                self.file_history_output])
        self.processing_step_id_dropdown = w.Dropdown(description = 'Processing step index:', 
                                                      style = {'description_width': 'initial'},
                                                      layout = {'width': '66%'})
        self.display_processing_step_details_button = w.Button(description = 'show detailed processing settings', 
                                                             style = {'description_width': 'initial'},
                                                             layout = {'width': '33%'})
        self.processing_step_details_output = w.Output()
        detailed_processing_history_widgets = w.VBox([w.HBox([self.processing_step_id_dropdown, self.display_processing_step_details_button]),
                                                      self.processing_step_details_output])
        browse_file_hostories_tab_widget = w.VBox([intro_text,
                                                   genereal_file_history_widgets,
                                                   detailed_processing_history_widgets])
        return browse_file_hostories_tab_widget
                                      
        
    def _bind_buttons_to_functions(self) -> None:
        self.update_project_files_button.on_click(self._update_project_files_button_clicked)
        self.display_current_project_files_button.on_click(self._display_current_project_files_button_clicked)
        self.save_project_button.on_click(self._save_project_button_clicked)
        self.load_project_button.on_click(self._load_project_button_clicked)
        self.display_file_history_button.on_click(self._display_file_history_button_clicked)
        self.confirm_microscopy_images_reader_settings_button.on_click(self._confirm_microscopy_images_reader_settings_button_clicked)
        self.confirm_rois_reader_settings_button.on_click(self._confirm_rois_reader_settings_button_clicked)
        self.display_processing_step_details_button.on_click(self._display_processing_step_details_button_clicked)
        
        
    def _confirm_rois_reader_settings_button_clicked(self, b) -> None:
        roi_reader_configs = self.rois_reader_specs.export_current_gui_config_values()
        self.api.set_roi_reader_configs(roi_reader_configs = roi_reader_configs)
        self.confirm_rois_reader_settings_button.description = 'refine settings'
        with self.displayed_output:
            print('ROI-file import settings successfully updated!')
        
    def _confirm_microscopy_images_reader_settings_button_clicked(self, b) -> None:
        microscopy_reader_configs = self.microscopy_images_reader_specs.export_current_gui_config_values()
        self.api.set_microscopy_reader_configs(microscopy_reader_configs = microscopy_reader_configs)
        self.confirm_microscopy_images_reader_settings_button.description = 'refine settings'
        with self.displayed_output:
            print('Microscopy image import settings successfully updated!')
        
        
    def _update_project_files_button_clicked(self, b) -> None:
        self.api.update_database_with_current_source_files()
        self.api.database.compute_file_infos()
        self._update_options_for_file_histories_id_dropdown()
        self._display_current_project_files_button_clicked('simulate a click')
        
    
    def _update_options_for_file_histories_id_dropdown(self) -> None:
        self.file_histories_id_dropdown.options = self.api.database.file_infos['file_id']
        
        
    def _display_current_project_files_button_clicked(self, b) -> None:
        if self.file_histories_id_dropdown.value == None:
            self._update_options_for_file_histories_id_dropdown()
        file_infos_df = pd.DataFrame(data = self.api.database.file_infos)
        refreshed_datetime = datetime.now()
        with self.current_project_files_output:
            self.current_project_files_output.clear_output()
            print(f'Data refreshed at {refreshed_datetime:%H:%M:%S} on {refreshed_datetime:%d.%m.%Y}')
            print('\n\n')
            display(file_infos_df)

            
    def _save_project_button_clicked(self, b) -> None:
        self.api.save_status()
    
    
    def _load_project_button_clicked(self, b) -> None:
        self.api.load_status()
        
    
    def _display_file_history_button_clicked(self, b) -> None:
        selected_file_id = self.file_histories_id_dropdown.value
        self._display_file_infos(file_id = selected_file_id)
        self._display_file_history(file_id = selected_file_id)
        self._update_processing_step_id_dropdown_options(file_id = selected_file_id)
        
        
    def _display_file_infos(self, file_id: str) -> None:
        file_infos = self.api.database.get_file_infos(file_id = file_id)
        file_infos_df = self._convert_dict_with_no_list_values_into_dataframe(dict_to_convert = file_infos)
        with self.file_infos_output:
            self.file_infos_output.clear_output()
            display(file_infos_df)
            
    def _convert_dict_with_no_list_values_into_dataframe(self, dict_to_convert: Dict) -> pd.DataFrame:
        for key, value in dict_to_convert.items():
            dict_to_convert[key] = [value]
        return pd.DataFrame(data = dict_to_convert)
        
        
    def _display_file_history(self, file_id: str) -> None:
        with self.file_history_output:
            self.file_history_output.clear_output()
            display(self.api.database.file_histories[file_id].tracked_history)
    
    
    def _update_processing_step_id_dropdown_options(self, file_id: str) -> None:
        processing_step_ids = list(self.api.database.file_histories[file_id].tracked_history.index.values)
        if len(processing_step_ids) > 0:
            self.processing_step_id_dropdown.options = processing_step_ids
            
            
    def _display_processing_step_details_button_clicked(self, b) -> None:
        file_id = self.file_histories_id_dropdown.value
        processing_step_id = self.processing_step_id_dropdown.value
        processing_step_settings = self.api.database.file_histories[file_id].tracked_settings[processing_step_id]
        processing_step_settings_df = self._convert_dict_with_no_list_values_into_dataframe(dict_to_convert = processing_step_settings)
        with self.processing_step_details_output:
            self.processing_step_details_output.clear_output()
            display(processing_step_settings_df)

In [ ]:
#| export

class ProcessingStepPage(PageButtonBundle):
    
        
    def _initialize_page_content(self) -> WidgetType:
        intro_html = w.HTML(value = ('<div style="font-size: 16px">'
                                     '<b>Processing methods: </b>'
                                     '</div>'
                                     'Please use the expandable widget(s) below to specify and configure the '
                                     f'methods of how you would like to run the {self.bundle_id} of your data. '
                                     'Please note that the processing methods will be run in the order they are '
                                     'displayed to you (top to bottom).'))
        self.exported_strategies_with_configs = []
        self._initialize_strategy_selection_accordion()
        self._initialize_processing_configs_widget()
        self._initialize_trigger_widget_elements()
        self._bind_buttons_to_functions()
        widget = w.VBox([GUI_SPACER,
                         intro_html, 
                         self.strat_selection_accordion,
                         GUI_SPACER,
                         self.processing_configs_widget,
                         GUI_SPACER,
                         w.HBox([self.file_ids_range, self.run])])
        return widget
    
    
    def _initialize_processing_configs_widget(self) -> None:
        processing_object_class = self.api.project_configs.available_processing_objects[self.bundle_id] 
        self.processing_obj = processing_object_class()
        self.processing_obj.initialize_gui_configs_and_widget()
        self.confirm_and_processing_configs = w.Button(description = 'confirm & export configurations',
                                                       layout = {'width': '30%'},
                                                       style = {'description_width': 'initial'})
        self.refine_processing_configs = w.Button(description = 'refine configurations',
                                                  disabled = True,
                                                  layout = {'width': '30%'},
                                                  style = {'description_width': 'initial'})
        self.processing_configs_widget = w.VBox([self.processing_obj.widget,
                                                 w.HBox([self.confirm_and_processing_configs, self.refine_processing_configs])])
        
        
    def _initialize_strategy_selection_accordion(self) -> None:
        available_strategy_classes = self.api.project_configs.available_processing_strategies[self.bundle_id]        
        self.strat_selection_accordion = w.Accordion()
        initial_strat_configurator = StrategyConfigurator(available_strategy_classes=available_strategy_classes,
                                                          parent_accordion=self.strat_selection_accordion,
                                                          target_for_configs_export=self.exported_strategies_with_configs)
        self.strat_selection_accordion.children = self.strat_selection_accordion.children + (initial_strat_configurator.widget, )
        self.strat_selection_accordion.set_title(0, 'Expand me to add a processing method')
        self.strat_selection_accordion.selected_index = None
        
        
    def _initialize_trigger_widget_elements(self) -> None:
        all_file_ids = self.api.database.file_infos['file_id']
        self.file_ids_range = w.SelectionRangeSlider(description = 'Select range of file IDs to process: ',
                                                     options = all_file_ids,
                                                     value = (all_file_ids[0], all_file_ids[-1]),
                                                     layout = {'width': '75%'},
                                                     style = {'description_width': 'initial'})
        self.run = w.Button(description = f'Launch {self.bundle_id}',
                            icon = 'rocket',
                            disabled = True,
                            layout = {'width': '25%'},
                            style = {'description_width': 'initial', 'button_color': 'orange'}
                           )

    
    def _bind_buttons_to_functions(self) -> None:
        self.confirm_and_processing_configs.on_click(self._confirm_and_processing_configs_clicked)
        self.refine_processing_configs.on_click(self._refine_processing_configs_clicked)
        self.run.on_click(self._run_clicked)
        
        
    def _confirm_and_processing_configs_clicked(self, b):
        self.processing_configs = self.processing_obj.gui_configs.export_current_config_values()
        self._change_disable_settings_of_customizable_widgets(disable_customizable_widgets = True)
        
    
    def _change_disable_settings_of_customizable_widgets(self, disable_customizable_widgets: bool) -> None:
        self.run.disabled = not disable_customizable_widgets
        self.refine_processing_configs.disabled = not disable_customizable_widgets
        self.confirm_and_processing_configs.disabled = disable_customizable_widgets
        for widget in self.processing_obj.widget.children:
            if hasattr(widget, 'disabled'):
                widget.disabled = disable_customizable_widgets
            elif type(widget) == w.HBox:
                if hasattr(widget.children[0], 'disabled'):
                    widget.children[0].disabled = disable_customizable_widgets
                    
                    
    def _refine_processing_configs_clicked(self, b) -> None:
        self.processing_configs = {}
        self._change_disable_settings_of_customizable_widgets(disable_customizable_widgets = False)
        
        
    def _run_clicked(self, b) -> None:
        self.run.disabled = True
        self.refine_processing_configs.disabled = True
        self._determine_and_call_corresponding_api_function()
        self.run.disabled = False
        self.refine_processing_configs.disabled = False
        
    
    def _determine_and_call_corresponding_api_function(self) -> None:
        file_ids = self._get_file_ids()
        strategies, strategy_configs = self._get_strategies_and_configs()
        if self.bundle_id == 'preprocessing':
            corresponding_api_callable = self.api.preprocess
        elif self.bundle_id == 'segmentation':
            corresponding_api_callable = self.api.segment
        elif self.bundle_id == 'postprocessing':
            corresponding_api_callable = self.api.postprocess
        elif self.bundle_id == 'quantification':
            corresponding_api_callable = self.api.quantify
        else:
            raise NotImplementedError(f'API wrapper for {self.bundle_id} missing!')
        with self.displayed_output:
            self.displayed_output.clear_output()
            corresponding_api_callable(strategies, strategy_configs, self.processing_configs, file_ids)
    
        
    def _get_file_ids(self) -> List[str]:
        all_file_ids = self.api.database.file_infos['file_id']
        start_idx = all_file_ids.index(self.file_ids_range.value[0])
        end_idx = all_file_ids.index(self.file_ids_range.value[1])
        file_ids = all_file_ids[start_idx : end_idx+1]
        return file_ids
    
    
    def _get_strategies_and_configs(self) -> Tuple[List[ProcessingStrategy], List[Dict]]:
        strategies, strategy_configs = zip(*self.exported_strategies_with_configs)
        strategies = list(strategies)
        strategy_configs = list(strategy_configs)
        return strategies, strategy_configs

In [ ]:
    def __init__(self, bundle_id: str, page_screen: WidgetType, all_navigator_buttons: List, api: API) -> None:
        self.bundle_id = bundle_id
        self.gui_page_screen = page_screen
        self.all_navigator_buttons = all_navigator_buttons
        self.api = api
        self.navigator_button = self._initialize_navigator_button()
        self.displayed_output = w.Output()
        self.page_content = self._initialize_page_content()

In [ ]:
#| export

class InspectionPage(PageButtonBundle):
    
    
    def _initialize_page_content(self) -> WidgetType:
        self.inspection_method_dropdown = w.Dropdown(description = 'Please select the inspection method you`d like to run:',
                                                     options = self.api.available_inspection_methods['option_strings_for_gui'],
                                                     layout = 'width': '100%',
                                                     style = {'description_width': 'initial'})
        self.file_id_selection_slider = w.SelectionSlider(description = 'File ID to inspect:', 
                                                          options = self.api.database.file_infos['file_id'],
                                                          layout = {'width': '50%'},
                                                          style = {'description_width': 'initial'})
        self.area_roi_id_dropdown = w.Dropdown(description = 'Area ROI ID to inspect:',
                                               options = ['Please select a fild ID first!'],
                                               layout = {'width': '35%'},
                                               style = {'description_width': 'initial'})
        self.confirm_and_load_or_reset_button = w.Button(description = 'confirm',
                                                         layout = {'width': '15%'},
                                                         tooltip = 'Depending on the image size, this might take a moment')
        self.confirm_method_file_and_area_roi_id_box = w.VBox([self.inspection_method_dropdown,
                                                              w.HBox([self.file_id_selection_slider,
                                                                      self.area_roi_id_dropdown, 
                                                                      self.confirm_and_load_or_reset_button])
                                                             ])
        self._bind_initial_button_and_event_handler()
        return self.confirm_method_file_and_area_roi_id_box
        
        
        
    def _bind_initial_button_and_event_handler(self) -> None:
        self.file_id_selection_slider.observe(self._update_area_roi_id_options, names='value')
        self.confirm_and_load_or_reset_button.on_click(self._confirm_and_load_or_reset_button_clicked)
        
        
    def _update_area_roi_id_options(self, change) -> None:
        selected_file_id = change['new']
        available_area_roi_ids = list(self.api.database.area_rois_for_quantification[selected_file_id]['all_planes'].keys())
        self.area_roi_id_dropdown.options = available_area_roi_ids
        
        
    def _confirm_and_load_or_reset_button_clicked(self, b) -> None:
        if self.confirm_and_load_or_reset_button.description == 'confirm':
            self.inspection_method_dropdown.disabled = True
            self.file_id_selection_slider.disabled = True
            self.area_roi_id_dropdown.disabled = True
            self.confirm_and_load_or_reset_button.description = 'reset'
            self.inspection_method_obj = self._initialize_inspection_method_object()
            self.extended_inspection_configs_widget = self._initialize_extended_inspection_configs_widget()
        else:
            if hasattr(self, 'extended_inspection_configs_widget') == True:
                delattr(self, 'extended_inspection_configs_widget')
            if hasattr(self, 'inspection_method_obj') == True:
                delattr(self, 'inspection_method_obj') 
            self.inspection_method_dropdown.disabled = False
            self.file_id_selection_slider.disabled = False
            self.area_roi_id_dropdown.disabled = False
            self.confirm_and_load_or_reset_button.description = 'confirm'
            # re-adjustment of parent children required?
            
            
    def _initialize_inspection_method_object(self) -> InspectionMethod:
        inspection_method_obj = self.api.initialize_inspection(inspection_method = self.inspection_method_dropdown.value,
                                                               file_id = self.file_id_selection_slider.value,
                                                               area_roi_id = self.area_roi_id_dropdown.value)
        inspection_method_obj.build_widget_for_remaining_conifgs()
        return inspection_method_obj
     

            
    def _initialize_extended_inspection_configs_widget(self) -> WidgetType:
        coords_selection_widgets = self._initalize_coords_selection_widgets()
        confirm_all_configs_and_run_inspection_button = w.Button(description = 'confirm all settings and run inspection', 
                                                                 icon = 'search', layout = {'width': '40%'})
        confirm_all_configs_and_run_inspection_button.on_click(self._confirm_all_configs_and_run_inspection_button_clicked)
        return w.VBox([coords_selection_widgets,
                       self.inspection_method_obj.widget,
                       confirm_all_configs_and_run_inspection_button])        
        
        
    def _confirm_all_configs_and_run_inspection_button_clicked(self) -> None:
        center_coords = (self.x_coord_text, self.y_coord_text)
        inspection_configs = self.inspection_method_obj.gui_configs.export_current_config_values()
        self.api.inspect(inspection_method_obj = self.inspection_method_obj,
                         center_coords = center_coords,
                         inspection_configs = inspection_configs)
        
        
    def _initalize_coords_selection_widgets(self) -> WidgetType:
        enter_prompt = w.HTML(value = ('Use one of the three methods (a-c) below to determine the center coordinates '
                                       'of the area you want to inspect & then enter them here:'), layout = {'width': '70%'})
        self.x_coord_text = w.Text(placeholder = 'x-coordinate', layout = {'width': '10%'})
        self.y_coord_text = w.Text(placeholder = 'y-coordinate', layout = {'width': '10%'})
        info_click = w.HTML(value = 'a) Open an overview image & determine coords by right-click', layout = {'width': '40%'})
        click_for_coords_button = w.Button(description = 'open overview',
                                           layout = {'width': '30%'})
        self.output_click = w.Output(layout = {'width': '30%'})
        label_id_dropdown = w.Dropdown(description = 'b) Select the unique ID of the feature you want to inspect:',
                                       options = self.inspection_method_obj.get_available_label_ids()
                                       layout = {'width': '70%'},
                                       style = {'description_width': 'initial'})
        self.output_label_id = w.Output(layout = {'width': '30%'})
        multi_match_idx_dropdown = w.Dropdown(description = 'c) Use the index of features listed in the multi-match-traceback:', 
                                              self.inspection_method_obj.get_available_multi_match_idxs()
                                               layout = {'width': '70%'},
                                               style = {'description_width': 'initial'})
        self.output_multi_match = w.Output(layout = {'width': '30%'})
        click_for_coords_button.on_click(self._click_for_coords_button_clicked)
        label_id_dropdown.observe(self._print_center_coords_based_on_label_id, names = 'value')
        multi_match_idx_dropdown.observe(self._print_center_coords_based_on_multi_match_idx, names = 'value')
        return w.VBox([w.HBox([enter_prompt, self.x_coord_text, self.y_coord_text]),
                       w.HBox([info_click, self.click_for_coords_button, self.output_click]),
                       w.HBox([label_id_dropdown, self.output_label_id]),
                       w.HBox([multi_match_idx_dropdown, self.output_multi_match])])
        
        
    def _click_for_coords_button_clicked(self) -> None:
        with self.output.click:
            self.output.click.clear_output()
            self.inspection_method_obj.get_center_coords_from_mouse_click_position()

            
    def _print_center_coords_based_on_label_id(self, change) -> None:
        x_coord, y_coord = self.inspection_method_obj.get_center_coords_from_label_id(label_id = change['new'])
        with self.output_label_id:
            self.output_label_id.clear_output()
            print(f'x: {x_coord}, and y: {y_coord}')
            
            
    def _print_center_coords_based_on_multi_match_idx(self, change) -> None:
        x_coord, y_coord = self.inspection_method_obj.get_center_coords_from_multi_match_idx(multi_match_idx = change['new'])
        with self.output_multi_match:
            self.output_multi_match.clear_output()
            print(f'x: {x_coord}, and y: {y_coord}')

In [ ]:
w.Button(description = 'a', icon = 'search')

Button(description='a', icon='search', style=ButtonStyle())

In [ ]:
info_text = w.HTML(value = 'a) Open an overview image & determine coords by right-click', layout = {'width': '40%'})
click_for_coords_button = w.Button(description = 'open overview',
                                   layout = {'width': '30%'})
output_click = w.Output(layout = {'width': '30%'})
click_box = w.HBox([info_text, click_for_coords_button, output_click])

In [ ]:
click_box

In [ ]:
label_id_dropdown = w.Dropdown(description = 'b) Select the unique ID of the feature you want to inspect:',
                               layout = {'width': '70%'},
                               style = {'description_width': 'initial'})
output_label_id = w.Output(layout = {'width': '30%'})
label_id_box = w.HBox([label_id_dropdown, output_label_id])

In [ ]:
label_id_box

In [ ]:
multi_match_idx_dropdown = w.Dropdown(description = 'c) Use the index of features listed in the multi-match-traceback:', 
                                       layout = {'width': '70%'},
                                       style = {'description_width': 'initial'})
output_multi_match = w.Output(layout = {'width': '30%'})
multi_match_box = w.HBox([multi_match_idx_dropdown, output_multi_match])

In [ ]:
multi_match_box

In [ ]:
enter_prompt = w.HTML(value = 'Use one of the three methods (a-c) below to determine the center coordinates of the area you want to inspect & then enter them here:', layout = {'width': '70%'})
x_coord_text = w.Text(placeholder = 'x-coordinate', layout = {'width': '10%'})
y_coord_text = w.Text(placeholder = 'y-coordinate', layout = {'width': '10%'})
enter_coords_box = w.HBox([enter_prompt, x_coord_text, y_coord_text])


In [ ]:
enter_coords_box

In [ ]:
delattr

In [ ]:
area_roi_id_dropdown = w.Dropdown(description = 'Area ROI ID to inspect:',
                                  options = ['Please select a fild ID first!'],
                                  layout = {'width': '35%'},
                                  style = {'description_width': 'initial'}
                                 )

In [ ]:
area_roi_ids = {'0000': [1, 2, 3], 
                '0001': [5, 6, 7]}

file_ids = ['0000', '0001']

In [ ]:
def _update_area_roi_ids(change) -> None:
    area_roi_id_dropdown.options = area_roi_ids[change['new']]

In [ ]:
def _confirm_and_load_or_reset_button_clicked(b) -> None:
    if confirm_and_load_or_reset_button.description == 'confirm':
        file_id_selection_slider.disabled = True
        area_roi_id_dropdown.disabled = True
        confirm_and_load_or_reset_button.description = 'reset'
    else:
        file_id_selection_slider.disabled = False
        area_roi_id_dropdown.disabled = False    
        confirm_and_load_or_reset_button.description = 'confirm'
        # del inspection object & all other widgets

In [ ]:
file_id_selection_slider = w.SelectionSlider(description = 'File ID to inspect:', 
                                              options = file_ids,
                                              layout = {'width': '50%'},
                                              style = {'description_width': 'initial'})
file_id_selection_slider.observe(_update_area_roi_ids, names = 'value')
confirm_and_load_or_reset_button = w.Button(description = 'confirm', layout = {'width': '15%'}, tooltip = 'Depending on the image size, this might take a moment')
box = w.HBox([file_id_selection_slider, area_roi_id_dropdown, confirm_and_load_or_reset_button])
confirm_and_load_or_reset_button.on_click(_confirm_and_load_or_reset_button_clicked)

In [ ]:
box

In [ ]:
#| export

class GUI:
    
    @property
    def _expected_processing_step_modules(self) -> List[str]:
        """
        This list defines the processing steps for which ProcessingStepPages will be created
        in the GUI version of findmycells. It will also be used to compare the elements of this
        list to the list of automatically detected processing modules, which can be found in 
        the "available_processing_modules" attribute of the ProcessingConfigs of the API. In case
        that this list of expected modules is missing an element, it will print a warning to alert
        the user / developer about this. The automatically created list could, unfortunately, not
        be used, as the order of ProcessingStepPages (and the corresponding navigator buttons)
        would not be correct.
        """
        return ['preprocessing', 'segmentation', 'postprocessing', 'quantification']
    
    def __init__(self) -> None:
        self.displayed_widget = self._initialize_start_screen()
        
    
    def _initialize_start_screen(self) -> WidgetType:
        welcome_html = w.HTML(value = ('<br><br>'
                                       '<div style="font-size: 26px" align="center">'
                                       '<b>Welcome to <i>findmycells</i> - glad you´re here! :-)</b>'
                                       '</div><br><br>'
                                       '<div style="font-size: 16px" align="center">'
                                       'Please start by selecting the root directory for your project below. '
                                       'Once you made your selection & are happy with it - click the "launch project" '
                                       'button to launch your project:'))        
        current_working_dir = os.getcwd()
        #self.root_dir_chooser = FileChooser(current_working_dir, show_only_dirs = True)
        self.root_dir_chooser = FileChooser('/mnt/c/Users/dsege/Downloads/fmc_test_project/', show_only_dirs = True)
        self.welcome_page_output = w.Output()
        confirm_root_dir_selection_button = w.Button(description = 'launch project', icon = 'rocket', layout = {'width': '25%'})
        confirm_root_dir_selection_button.on_click(self._confirm_root_dir_selection)
        return w.VBox([welcome_html, self.root_dir_chooser, confirm_root_dir_selection_button, self.welcome_page_output],
                      layout = {'align_items': 'center'})
   
    
    def _confirm_root_dir_selection(self, b) -> None:
        if self.root_dir_chooser.value == None:
            with self.welcome_page_output:
                self.welcome_page_output.clear_output()
                print(('Whoooops - seems like you have not yet made your selection! '
                       'This requires you to click twice on the "select" button. Once '
                       'to open the file explorer widget, and a second time to collapse '
                       'it again. After your second click, you should see the path you '
                       'selected displayed above in green. Once this is the case, please '
                       'click the "launch project" button again.'))
        else:
            selected_root_dir_path = Path(self.root_dir_chooser.value)
            self.api = API(project_root_dir = selected_root_dir_path)
            self._initialize_main_screen()
        
        
    def _initialize_main_screen(self) -> None:
        self.page_screen = w.VBox()
        self.api.update_database_with_current_source_files()
        self._initialize_page_bundles()
        navigator_bar = w.HBox(self.navigator_buttons, layout = {'align_items': 'center'})
        self.main_screen = w.VBox([navigator_bar, self.page_screen], layout = {'width': '100%'})
        self._refresh_displayed_widget(new_widget = self.main_screen)
        self.navigator_buttons[0].click() # simulate a click on settings page'
        
        
    def _initialize_page_bundles(self) -> None:
        self.navigator_buttons = []
        self.settings_page = SettingsPage(bundle_id = 'settings',
                                          page_screen = self.page_screen,
                                          all_navigator_buttons = self.navigator_buttons,
                                          api = self.api)
        self.navigator_buttons.append(self.settings_page.navigator_button)
        # ToDo: add reader configs here?
        self._compare_expected_to_available_processing_modules()
        for processing_step_module in self._expected_processing_step_modules:
            processing_step_page = ProcessingStepPage(bundle_id = processing_step_module,
                                                      page_screen = self.page_screen,
                                                      all_navigator_buttons = self.navigator_buttons,
                                                      api = self.api)
            self.navigator_buttons.append(processing_step_page.navigator_button)
            attr_id = f'{processing_step_module}_page'
            setattr(self, attr_id, processing_step_page)

            
    def _compare_expected_to_available_processing_modules(self) -> None:
        available_processing_modules = list(self.api.project_configs.available_processing_modules.keys())
        for available_module in available_processing_modules:
            if available_module not in self._expected_processing_step_modules:
                print(f'Warning for developers: {available_module} is an available processing module of findmycells,'
                       'which is not yet covered in the GUI! Please also add it to the "_expected_processing_step_modules"'
                       'property of the GUI class in findmycells.interfaces.')
        
    def _refresh_displayed_widget(self, new_widget: WidgetType) -> None:
        self.displayed_widget.children = (new_widget, )

In [ ]:
#| hide

gui = GUI()
gui.displayed_widget

In [ ]:
#| export

from typing import Tuple, List, Dict, Any, Optional
from pathlib import Path
from abc import ABC, abstractmethod

import numpy as np
from skimage import io, color

from findmycells.database import Database
from findmycells import utils
from findmycells.configs import DefaultConfigs, GUIConfigs

from matplotlib.backend_bases import MouseButton
import matplotlib.pyplot as plt

In [ ]:
#| export

class InspectionMethod(ABC):
    
    @abstractmethod
    def _initialize_default_configs(self) -> DefaultConfigs:
        pass
    

    @abstractmethod
    def _initialize_gui_configs(self) -> GUIConfigs:
        pass
    
    
    @abstractmethod
    def _method_specific_inspection(self, center_pixel_coords: Tuple[int, int], inspection_configs: Dict[str, Any]) -> None:
        pass
    
    
    def __init__(self, file_id: str, area_roi_id: str, database: Database) -> None:
        self.file_id = file_id
        self.area_roi_id = area_roi_id
        self.database = database
        self.preprocessed_image = self._load_preprocessed_image()
        self.postprocessed_segmentation_mask = self._load_postprocessed_segmentation_mask()
        self.rgb_color_coded_2d_overlay_of_image_and_mask = self._create_rgb_color_coded_2d_overlay_of_image_and_mask()
        self.area_roi_boundary_coords = self._load_area_roi_boundary_coords()
        self.default_configs = self._initialize_default_configs()
        self.gui_configs = self._initialize_gui_configs()

    
    def _load_preprocessed_image(self) -> np.ndarray:
        preprocessed_images_dir_path = self.database.project_configs.root_dir.joinpath(self.database.preprocessed_images_dir)
        return utils.load_zstack_as_array_from_single_planes(path = preprocessed_images_dir_path, file_id = self.file_id)


    def _load_postprocessed_segmentation_mask(self) -> np.ndarray:
        postprocessed_masks_dir_path = self.database.project_configs.root_dir.joinpath(self.database.quantified_segmentations_dir, self.area_roi_id)
        return utils.load_zstack_as_array_from_single_planes(path = postprocessed_masks_dir_path, file_id = self.file_id)
    
      
    def _create_rgb_color_coded_2d_overlay_of_image_and_mask(self) -> np.ndarray:
        max_projection_image = np.max(self.preprocessed_image, axis=0)
        max_projection_mask = np.max(self.postprocessed_segmentation_mask, axis=0)
        return color.label2rgb(max_projection_mask, image = max_projection_image, bg_label = 0, bg_color = None, saturation = 1, alpha = 1)     
    

    def _load_area_roi_boundary_coords(self) -> Tuple[np.ndarray, np.ndarray]:
        y_coords, x_coords = self.database.area_rois_for_quantification[self.file_id]['all_planes'][self.area_roi_id].boundary.coords.xy
        return np.asarray(x_coords, dtype='int'), np.asarray(y_coords, dtype='int')
    
    
    def get_available_label_ids(self) -> List[int]:
        return list(np.unique(self.postprocessed_segmentation_mask))
    
    
    def get_available_multi_match_idxs(self) -> List[int]:
        return self.database.multi_matches_traceback[self.file_id]['final_label_id']
    
    
    def get_center_coords_from_label_id(self, label_id: int) -> Tuple[int, int]:
        if self.postprocessed_segmentation_mask.shape[0] > 1:
            mask_as_single_plane = np.max(self.postprocessed_segmentation_mask, axis=0)
        else:
            mask_as_single_plane = self.postprocessed_segmentation_mask
        feature_roi = utils.get_polygon_from_instance_segmentation(single_plane = mask_as_single_plane, label_id = label_id)
        return (feature_roi.centroid.y, feature_roi.centroid.x)


    def get_center_coords_from_multi_match_idx(self, multi_match_idx: int) -> Tuple[int, int]:
        label_id = self.database.multi_matches_traceback[self.file_id]['final_label_id'][multi_match_idx]
        return self.get_center_coords_from_label_id(label_id = label_id)

    
    def get_center_coords_from_mouse_click_position(self) -> Tuple[int, int]:
        self._check_for_matplotlib_setup()
        fig = plt.figure(figsize=(10, 10), facecolor = 'white')
        plt.connect('button_press_event', self._matplotlib_figure_clicked)
        plt.imshow(self.rgb_color_coded_2d_overlay_of_image_and_mask)
        plt.plot(self.area_roi_boundary_coords[0], self.area_roi_boundary_coords[1], c = 'cyan')
        plt.show()
        
        
    def _check_for_matplotlib_setup(self) -> None:
        if hasattr(self, 'matplotlib_all_set_up') == False:
            from IPython import get_ipython
            ipy = get_ipython()
            if ipy is not None:
                ipy.run_line_magic('matplotlib', 'tk')
            self.matplotlib_all_set_up = True
        else:
            pass

        
    def _matplotlib_figure_clicked(self, event):
        if event.button is MouseButton.RIGHT:
            # only alternative to printing the values (without installing additional packages) seems to be a global variable...
            print(f'Your selected x-coordinate was: {event.x}, and your selected y-coordinate was: {event.y}.')
            plt.close()
            
        
    def build_widget_for_remaining_conifgs(self) -> None:
        info_text = 'Additional configs for your inspection plot:'
        self.gui_configs.construct_widget(info_text = info_text, default_configs = inspection_method.default_configs)
        self.widget = self.gui_configs.strategy_widget
   
    
    def run_inspection(self, center_pixel_coords: Tuple[int, int], inspection_configs: Optional[Dict[str, Any]]=None) -> None:
        self._check_for_matplotlib_setup()
        inspection_configs = self._validate_and_update_inspection_configs(inspection_configs = inspection_configs)
        self._method_specific_inspection(center_pixel_coords = center_pixel_coords, inspection_configs = inspection_configs)
        
        
    def _validate_and_update_inspection_configs(self, inspection_configs: Optional[Dict[str, Any]]) -> Dict[str, Any]:
        if type(inspection_configs) != dict:
            inspection_configs = self.default_configs.values
        else:
            self.default_configs.assert_user_input(user_input = inspection_configs)
            inspection_configs = self.default_configs.fill_user_input_with_defaults_where_needed(user_input = inspection_configs)
        return inspection_configs
    

In [ ]:
class InspectStackIn3D(InspectionMethod):

    
    def _initialize_default_configs(self) -> DefaultConfigs:
        max_box_size = min(self.rgb_color_coded_2d_overlay_of_image_and_mask.shape[0:2])
        box_size, binning_factor = self._get_default_box_size_and_binning_factor(max_box_size = max_box_size)
        default_values = {'box_size': box_size,
                          'binning_factor': binning_factor,
                          'show': True,
                          'save': False}
        valid_types = {'box_size': [int],
                       'binning_factor': [int],
                       'show': [bool], 
                       'save': [bool]}
        valid_ranges = {'box_size': (50, max_box_size, 50),
                        'binning_factor': (0, 100, 1)}
        default_configs = DefaultConfigs(default_values = default_values,
                                         valid_types = valid_types,
                                         valid_value_ranges = valid_ranges)
        return default_configs
    
    
    def _initialize_gui_configs(self) -> GUIConfigs:
        descriptions = {'box_size': 'Specify the size of the area that you would like to inspect [px]:',
                        'binning_factor': 'We recommend to select a binning factor such that: box size / binning factor ~ 50 (no remainder allowed!):',
                        'show': 'show the plot',
                        'save': 'save the resulting plot'}
        widget_names = {'box_size': 'IntSlider',
                        'binning_factor': 'IntSlider',
                        'show': 'Checkbox',
                        'save': 'Checkbox'}
        return GUIConfigs(widget_names = widget_names, descriptions = descriptions)
    
    
    def _get_default_box_size_and_binning_factor(self, max_box_size: int) -> Tuple[int, int]:
        size_factor = max_box_size // 500
        default_box_size = size_factor * 100
        if default_box_size == 0:
            default_box_size = min(max_box_size, 100)
        if default_box_size < 100:
            default_binning_factor = 0
        else:
            default_binning_factor = default_box_size // 50
        return default_box_size, default_binning_factor
    
    
    
    def _method_specific_inspection(self,
                                    center_pixel_coords: Tuple[int, int],
                                    inspection_configs: Dict[str, Any]
                                   ) -> None:
        cropping_boundaries = self._calculate_cropping_boundaries(center_coords = center_pixel_coords, inspection_configs = inspection_configs)
        cropped_and_binned_mask_zstack = self._crop_and_bin_zstack(cropping_boundaries = cropping_boundaries, inspection_configs = inspection_configs)
        voxel_color_code = color.label2rgb(cropped_and_binned_mask_zstack)
        self._create_3d_plot(box_boundaries = cropping_boundaries, 
                             voxels = cropped_and_binned_mask_zstack, 
                             color_code = voxel_color_code, 
                             inspection_configs = inspection_configs)
       
    

    def _calculate_cropping_boundaries(self, center_coords: Tuple[int, int], inspection_configs: Dict[str, Any]) -> Dict:
        center_row, center_col = center_coords
        max_row, max_col = self.rgb_color_coded_2d_overlay_of_image_and_mask.shape[:2]
        lower_row_cropping_idx, upper_row_cropping_idx = self._determine_cropping_indices(center_px_idx = center_row, 
                                                                                          half_box_size = inspection_configs['box_size']//2, 
                                                                                          max_idx = max_row)
        lower_col_cropping_idx, upper_col_cropping_idx = self._determine_cropping_indices(center_px_idx = center_col,
                                                                                          half_box_size = inspection_configs['box_size']//2,
                                                                                          max_idx = max_col)
        cropping_boundaries = {'lower_row': lower_row_cropping_idx,
                               'upper_row': upper_row_cropping_idx,
                               'lower_col': lower_col_cropping_idx,
                               'upper_col': upper_col_cropping_idx}
        return cropping_boundaries
    
    
    def _determine_cropping_indices(self, center_px_idx: int, half_box_size: int, max_idx: int) -> Tuple[int, int]:
        if (center_px_idx - half_box_size >= 0) & (center_px_idx + half_box_size <= max_idx):
            lower_cropping_index, upper_cropping_index = center_px_idx - half_box_size,  center_px_idx + half_box_size
        elif 2*half_box_size <= max_idx:
            if center_px_idx - half_box_size < 0:
                lower_cropping_index, upper_cropping_index = 0,  0 + 2*half_box_size
            else: # means: center_px_idx + half_box_size > max_index
                lower_cropping_index, upper_cropping_index = max_idx - 2*half_box_size, max_idx
        else:
            raise ValueError((f'The desired box size {2*half_box_size}) is larger than one of the '
                              f'image axes ({max_idx}). Please select a smaller "box_size"!'))
        return lower_cropping_index, upper_cropping_index
    

    def _crop_and_bin_zstack(self, cropping_boundaries: Dict, inspection_configs: Dict[str, Any]) -> np.ndarray:
        zstack = self.postprocessed_segmentation_mask
        cropped_zstack = zstack[:,
                                cropping_boundaries['lower_row']:cropping_boundaries['upper_row'],
                                cropping_boundaries['lower_col']:cropping_boundaries['upper_col']].copy()
        if inspection_configs['binning_factor'] > 0:
            cropped_and_binned_zstack = self._bin_zstack(zstack_to_bin = cropped_zstack, binning_factor = inspection_configs['binning_factor'])
        else:
            cropped_and_binned_zstack = cropped_zstack
        return cropped_and_binned_zstack
    
    
    def _bin_zstack(self, zstack_to_bin: np.ndarray, binning_factor: int) -> np.ndarray:
        zstack, new_shape = self._adjust_zstack_for_binning_to_new_shape(input_zstack = zstack_to_bin, binning_factor = binning_factor)
        binned_single_planes = []
        for plane_index in range(zstack.shape[0]):
            binned_single_planes.append(self._bin_2d_image(single_plane = zstack[plane_index], new_shape = new_shape))
        return np.asarray(binned_single_planes)
    
    
    def _adjust_zstack_for_binning_to_new_shape(self, input_zstack: np.ndarray, binning_factor: int) -> Tuple[np.ndarray, Tuple[int, int]]:
        zstack = input_zstack.copy()
        rows, cols = zstack[0].shape[:2]
        if rows % binning_factor != 0:
            excess_pixels = rows % binning_factor
            cropping_lower_end = int(excess_pixels/2)
            cropping_upper_end = rows - (excess_pixels - cropping_lower_end)
            zstack = zstack[:, cropping_lower_end:cropping_upper_end, :]
        if cols % binning_factor != 0:
            excess_pixels = cols % binning_factor
            cropping_lower_end = int(excess_pixels/2)
            cropping_upper_end = cols - (excess_pixels - cropping_lower_end)
            zstack = zstack[:, :, cropping_lower_end:cropping_upper_end]
        adjusted_rows, adjusted_cols = zstack[0].shape[:2]
        if (adjusted_rows % binning_factor != 0) or (adjusted_cols % binning_factor != 0):
            raise ValueError(('Sorry! Something went wrong during the binning process. To avoid this issue, '
                              'consider changing the "box_size" or the "binning_factor", such that '
                              'division of the box size by the binning factor does not yield any remainder!'))
        else:
            new_shape = (int(adjusted_rows / binning_factor), int(adjusted_cols / binning_factor))
        return zstack, new_shape
    
    
    def _bin_2d_image(self, single_plane: np.ndarray, new_shape: Tuple[int, int]):
        shape = (new_shape[0], single_plane.shape[0] // new_shape[0],
                 new_shape[1], single_plane.shape[1] // new_shape[1])
        return single_plane.reshape(shape).max(-1).max(1)
    

    def _create_3d_plot(self, box_boundaries: Dict, voxels: np.ndarray, color_code: np.ndarray, inspection_configs: Dict[str, Any]) -> None:
        
        box_size = box_boundaries['upper_row'] - box_boundaries['lower_row']
        center_row_index = box_boundaries['lower_row'] + 0.5*inspection_configs['box_size']
        center_col_index = box_boundaries['lower_col'] + 0.5*inspection_configs['box_size']
        box_row_coords = [box_boundaries['lower_row'], box_boundaries['lower_row'],
                          box_boundaries['upper_row'], box_boundaries['upper_row'], 
                          box_boundaries['lower_row']]
        box_col_coords = [box_boundaries['lower_col'], box_boundaries['upper_col'], 
                          box_boundaries['upper_col'], box_boundaries['lower_col'], 
                          box_boundaries['lower_col']]

        fig = plt.figure(figsize=(12,8), facecolor='white')
        gs = fig.add_gridspec(1,3)

        fig.add_subplot(gs[0,0])
        plt.imshow(self.rgb_color_coded_2d_overlay_of_image_and_mask)
        plt.plot(self.area_roi_boundary_coords[0], self.area_roi_boundary_coords[1])
        plt.plot(box_col_coords, box_row_coords, c='magenta', lw='3', linestyle='dashed')

        ax1 = fig.add_subplot(gs[0,1], projection='3d')
        ax1.voxels(voxels, facecolors=color_code)
        ax1.view_init(elev=30, azim=33)

        ax2 = fig.add_subplot(gs[0,2], projection='3d')
        ax2.voxels(voxels, facecolors=color_code)
        ax2.view_init(elev=30, azim=213)

        plt.suptitle((f'Area_id {self.area_roi_id} of file_id {self.file_id}, '
                      f'centered at ({center_row_index}, {center_col_index}) '
                      f'with a binning of factor {inspection_configs["binning_factor"]}'), y=0.9)
        if inspection_configs['save'] == True:
            filename = f'{self.file_id}_3D_inspection_of_{self.area_roi_id}.png'
            filepath = self.database.project_configs.root_dir.joinpath(self.database.inspection_dir, filename)
            plt.savefig(filepath, dpi=300)
        if inspection_configs['show'] == True:
            plt.show()
        else:
            plt.close()   

In [ ]:
inspection_method = InspectStackIn3D(file_id = '0000', area_roi_id = '#5_SN_tg_s2_sfr3', database = gui.api.database)

In [ ]:
center_coords = (3121, 1570)
inspection_configs = {'box_size': 750,
                      'binning_factor': 15}

In [ ]:
inspection_method.run_inspection(center_pixel_coords = center_coords, inspection_configs = inspection_configs)

In [ ]:
inspection_method.get_center_coords_from_mouse_click_position()

In [ ]:
inspection_method.get_center_coords_from_label_id(2047)

(1570.965112075199, 3121.530730296457)

In [ ]:
inspection_method.default_configs.fill_user_input_with_defaults_where_needed

In [ ]:
a = inspection_method.gui_configs.construct_widget(info_text = 'Optional configs:', default_configs = inspection_method.default_configs)

In [ ]:
a = inspection_method.gui_configs.strategy_widget

In [ ]:
a

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()